In [36]:
import THESIS2019.to_lexicon as lex
from base_words import *
import datetime
import pickle

import gensim
from gensim.models.wrappers import DtmModel
from gensim.models import LdaSeqModel
from gensim.corpora import Dictionary, bleicorpus
import os, numpy

In [26]:
path = "/Users/ninawang/Thesis/remote/THESIS2019/NYT-OPINION2012-2013-processed/"

start = datetime.datetime(2012, 6, 1)
end = datetime.datetime(2013, 7, 1)

articles2012 = lex.get_articles_from_filepath(path,start,end)

path = "/Users/ninawang/Thesis/remote/THESIS2019/NYT-OPINION2016-2017-processed/"

start = datetime.datetime(2016, 6, 1)
end = datetime.datetime(2017, 7, 1)

articles2016 = lex.get_articles_from_filepath(path,start,end)

print("Gathered %d articles from 2012" %len(articles2012))
print("Gathered %d articles from 2016" %len(articles2016))

Gathered 275 articles from 2012
Gathered 5971 articles from 2016


In [27]:
dem2012, rep2012 = lex.get_collocations(articles2012, LEFT_WORDS, RIGHT_WORDS)
dem2016, rep2016 = lex.get_collocations(articles2016, LEFT_WORDS, RIGHT_WORDS)

In [34]:
#### ! DUMP ! ######
# with open("2016-collocations-dem.pkl", "wb") as f:
#     pickle.dump(dem2016, f)
# with open("2016-collocations-rep.pkl", "wb") as f:
#     pickle.dump(rep2016, f)

In [41]:
def get_doc_and_timeslices(col):
    total = [d for d in col if d[0] is not None]
    docs = [d[1] for d in total]
    timeslices = [d[0].year for d in total]
    return docs, timeslices

dems_docs_2012, dems_timeslices_2012 = get_doc_and_timeslices(dem2012)
dems_docs_2016, dems_timeslices_2016 = get_doc_and_timeslices(dem2016)
combined = dems_docs_2012 + dems_docs_2016

dct = Dictionary(combined)
docs = []
for text in combined:
    docs.append(dct.doc2bow(text))
    
my_corpus = docs
my_timeslices = [len(dems_docs_2012),len(dems_docs_2016)]
exe = "../dtm-darwin64"


In [43]:
print(len(dems_docs_2012))
print(len(dems_docs_2016))

99
1921


In [44]:
model = DtmModel(exe, my_corpus, my_timeslices, num_topics=15, id2word=dct)

In [46]:
#### ! DUMP ! ######
# with open("sample-dtm.pkl", "wb") as f:
#     pickle.dump(model, f)

In [48]:
model.print_topics(num_topics=-1,num_words=5)

['0.086*right + 0.072*gun + 0.057*abort + 0.050*justic + 0.048*issu',
 '0.025*women + 0.012*white + 0.009*men + 0.008*work + 0.007*say',
 '0.038*immigr + 0.028*union + 0.021*european + 0.017*europ + 0.016*right',
 '0.023*vote + 0.016*voter + 0.014*elect + 0.013*state + 0.010*parti',
 '0.223*russia + 0.133*lie + 0.119*order + 0.119*putin + 0.059*hack',
 '0.037*presid + 0.016*black + 0.011*year + 0.010*first + 0.009*nation',
 '0.204*sign + 0.136*opinion + 0.120*newslett + 0.045*warren + 0.036*articl',
 '0.034*parti + 0.024*polit + 0.019*social + 0.011*cultur + 0.010*peopl',
 '0.013*get + 0.011*say + 0.008*make + 0.008*peopl + 0.008*time',
 '0.012*presid + 0.007*year + 0.007*care + 0.007*administr + 0.007*health',
 '0.166*continu + 0.157*read + 0.148*main + 0.146*stori + 0.144*advertis',
 '0.017*email + 0.011*investig + 0.010*presid + 0.009*report + 0.009*comey',
 '0.018*state + 0.016*presid + 0.014*unit + 0.011*american + 0.011*administr',
 '0.064*york + 0.063*new + 0.061*time + 0.047*vi

In [ ]:
import time
start_time = time.time()

ldaseq = LdaSeqModel(corpus=my_corpus, id2word=dct, time_slice=my_timeslices, num_topics=15)

print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/lib/python3.7/site-packages/gensim/models/ldaseqmodel.py:293: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
